In [1]:
# RFM ile müşteri segmentasyonu
# RFM: Recency, Frequency, Monetary
# Recency: Son alışveriş tarihine göre müşteri segmentasyonu
# Frequency: Müşterinin alışveriş sıklığına göre segmentasyonu
# Monetary: Müşterinin harcama miktarına göre segmentasyonu

In [2]:
# 1. iş problemi (Business Problem)
# 2. veriyi anlama (Data Understanding)
# 3. veriyi hazırlama (Data Preparation)
# 4. RFM Metriklerini Hesaplama  (calculate RFM metrics)
# 5. RFM Skorlarını Hesaplama (calculate RFM scores)
# 6. RFM segmentlerinin oluşturulması ve analiz edilmesi (create and analyze RFM segments)
# 7. Tüm sürecin fonksiyonlaştırılması (functionalization of the entire process)

In [ ]:
# 1. İŞ PROBLEMİ
# Bir e-ticaret şirketi, müşteri segmentasyonu yaparak pazarlama stratejilerini geliştirmek istiyor.

In [6]:
!pip install matplotlib

  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl (8.1 MB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\furka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#kütüphaneler
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\furka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\furka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [13]:
df_ = pd.read_excel("Kurs Materyalleri(CRM Analitiği)\datasets\online_retail_II.xlsx")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [14]:
df.shape

(525461, 8)

In [15]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [16]:
df["Description"].nunique() #ürün sayısı

4681

In [17]:
df["Description"].value_counts().head(10) #en çok satılan ürünler

Description
WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
60 TEATIME FAIRY CAKE CASES           1400
HOME BUILDING BLOCK WORD              1386
JUMBO BAG RED RETROSPOT               1310
LUNCH BAG RED SPOTTY                  1274
REX CASH+CARRY JUMBO SHOPPER          1232
Name: count, dtype: int64

In [ ]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head(10) #ürün bazında satış miktarları

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925
60 TEATIME FAIRY CAKE CASES,36326
PACK OF 60 PINK PAISLEY CAKE CASES,31822
JUMBO BAG RED RETROSPOT,30727
SMALL POPCORN HOLDER,29500


In [ ]:
df["Invoice"].nunique() #fatura sayısı

Invoice
537434    675
538071    652
537638    601
537237    597
536876    593
536592    592
537823    591
536031    582
490074    580
491966    579
Name: count, dtype: int64

In [21]:
df["Total_price"] = df["Quantity"] * df["Price"]  

In [22]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [23]:
df.groupby("Invoice").agg({"Total_price": "sum"}).sort_values("Total_price", ascending=False).head(10) #fatura bazında toplam satış fiyatları

,Total_price
Invoice,
533027,49844.990
531516,45332.970
493819,44051.600
524181,33167.800
526934,26007.080
512771,25111.090
515944,22863.360
503061,22853.770
517731,21984.000
